# Imports

In [1]:
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.tabular.all import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import random
from tqdm import tqdm
from ipywidgets import interact

from fastai.imports import *
np.set_printoptions(linewidth=130)


from pathlib import Path
import os


from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import roc_auc_score,accuracy_score,mean_squared_error, matthews_corrcoef
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score



#transformers and pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn import set_config

import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier

import lightgbm as lgb
from lightgbm import LGBMClassifier

from catboost import CatBoostClassifier,CatBoostRegressor,Pool, metrics, cv





import optuna
from optuna.samplers import TPESampler
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice
from optuna.samplers import TPESampler
import warnings


matplotlib.rc('image', cmap='Greys')

from fastkaggle import setup_comp



from openfe import OpenFE, transform
from autogluon.tabular import TabularDataset, TabularPredictor

import h2o
from h2o.automl import H2OAutoML

import gc

from xgboost import plot_importance
import wandb
from fastai.callback.wandb import *
from wandb.integration.lightgbm import wandb_callback, log_summary
#from wandb.xgboost import wandb_callback



#from IPython.display import FileLink

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
path = Path('playground-series-s4e8/')
path

Path('playground-series-s4e8')

In [3]:
train_df = pd.read_csv(path/'train.csv',index_col='id')
test_df = pd.read_csv(path/'test.csv',index_col='id')
sub_df = pd.read_csv(path/'sample_submission.csv')

In [4]:
!ls

AutogluonModels  playground-series-s4e8		     submission.csv
lgbm_model.pkl	 playground-series-s4e8.zip	     wandb
main.py		 poisonous_mushrooms_classification  xgb_model.json
models		 requirements.txt


# intergrating fastai

In [5]:
# Your existing FastAI preprocessing code
train_df = pd.read_csv(path/'train.csv', index_col='id')
test_df = pd.read_csv(path/'test.csv', index_col='id')
sub_df = pd.read_csv(path/'sample_submission.csv')

# Make a copy of the training dataframe to avoid modifying the original
train_df_with_nan = train_df.copy()

# Add a NaN value to a random row in the 'stem-height' column
random_index = np.random.choice(train_df_with_nan.index)
train_df_with_nan.loc[random_index, 'stem-height'] = np.nan

cont_names, cat_names = cont_cat_split(train_df_with_nan, dep_var='class')
splits = RandomSplitter(valid_pct=0.2)(range_of(train_df_with_nan))
to = TabularPandas(train_df_with_nan, procs=[Categorify, FillMissing, Normalize],
                   cat_names=cat_names,
                   cont_names=cont_names,
                   y_names='class',
                   y_block=CategoryBlock(),
                   splits=splits)

# Extract processed data from FastAI TabularPandas
train_processed = to.train.xs
train_processed['class'] = to.train.ys.values.flatten()

valid_processed = to.valid.xs
valid_processed['class'] = to.valid.ys.values.flatten()

full_processed = pd.concat([train_processed, valid_processed], axis=0)

# Use AutoGluon with the processed data
# from autogluon.tabular import TabularPredictor
# predictor = TabularPredictor(label='class').fit(train_data=full_processed)

# Process test data using FastAI's methods
test_dl = to.test_dl(test_df)
test_processed = test_dl.xs

# Now you can use test_processed with AutoGluon
# predictions = predictor.predict(test_processed)

AttributeError: test_dl

In [9]:
# Your existing FastAI preprocessing code
train_df = pd.read_csv(path/'train.csv', index_col='id')
test_df = pd.read_csv(path/'test.csv', index_col='id')
sub_df = pd.read_csv(path/'sample_submission.csv')

# Make a copy of the training dataframe to avoid modifying the original
train_df_with_nan = train_df.copy()

# Add a NaN value to a random row in the 'stem-height' column
random_index = np.random.choice(train_df_with_nan.index)
train_df_with_nan.loc[random_index, 'stem-height'] = np.nan

cont_names, cat_names = cont_cat_split(train_df_with_nan, dep_var='class')
splits = RandomSplitter(valid_pct=0.2)(range_of(train_df_with_nan))
to = TabularPandas(train_df_with_nan, procs=[Categorify, FillMissing, Normalize],
                   cat_names=cat_names,
                   cont_names=cont_names,
                   y_names='class',
                   y_block=CategoryBlock(),
                   splits=splits)

# Extract processed data from FastAI TabularPandas
train_processed = to.train.xs
train_processed['class'] = to.train.ys.values.flatten()

valid_processed = to.valid.xs
valid_processed['class'] = to.valid.ys.values.flatten()

full_processed = pd.concat([train_processed, valid_processed], axis=0)

# Process test data using FastAI's methods
test_processed = to.new(test_df)

# Use AutoGluon with the processed data
# from autogluon.tabular import TabularPredictor
# predictor = TabularPredictor(label='class').fit(train_data=full_processed)
# predictions = predictor.predict(test_processed)


In [ ]:
target = 'class'
TIME_LIMIT = 3600 * 10
predictor = TabularPredictor(
    label=target,
    eval_metric='mcc',
    problem_type='binary',
    verbosity=2
)

: 

In [ ]:
predictor.fit(
    train_data=full_processed,
    time_limit=TIME_LIMIT,
    presets='best_quality',
    excluded_model_types=['KNN'],
    ag_args_fit={
        #'num_gpus': 1, 
        'stopping_metric': 'log_loss'
    }
)

: 

In [ ]:
full_processed.shape,train_df_with_nan.shape

((3116945, 23), (3116945, 21))

In [ ]:
full_processed

,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season,cap-diameter_na,stem-height_na,cap-diameter,stem-height,stem-width,class
id,,,,,,,,,,,,,,,,,,,,,,,
2705509,68,48,75,9,45,32,60,0,41,56,0,0,6,19,0,30,3,1,1,-0.708218,-0.984961,-1.051938,1
2421674,72,57,65,9,45,29,48,0,0,36,0,0,6,19,0,38,4,1,1,-0.671741,-0.892347,-0.570068,0
1010942,54,0,64,9,76,29,60,30,44,56,0,0,6,19,0,31,3,1,1,0.737980,1.204433,0.020531,1
1420876,72,73,65,9,67,0,47,0,0,47,0,0,6,19,0,26,1,1,1,-0.433569,-0.258867,0.024238,0
2393120,72,60,62,9,45,0,60,0,0,56,0,0,6,19,23,30,1,1,1,-0.916350,-1.214643,-1.096419,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971773,68,77,66,21,48,29,48,0,0,56,0,0,6,19,0,26,3,1,1,0.583490,0.281998,0.726038,0
742987,72,0,64,9,48,29,60,16,0,47,0,0,6,19,0,26,1,1,1,-0.122443,-0.207004,-0.124030,1
2643456,49,73,77,9,0,29,50,0,0,58,20,22,19,40,24,26,1,1,1,-0.349887,1.119228,-0.330369,1


In [ ]:
%%time
#predictions = predictor.predict(test_processed)
predictions = predictor.predict(test_df)

: 

In [ ]:
%%time
predictor.evaluate(train_df)

: 

In [ ]:
%%time
predictor.leaderboard(train_df)

: 

In [ ]:
#submit = pd.read_csv(path/'sample_submission.csv')
!rm submission.csv
predictions = predictions.reset_index(drop=False)
submit = pd.DataFrame({'id': predictions['id'], 'class': predictions['class']})
submit.set_index('id', inplace=True)
submit.to_csv('submission.csv', index=True)
sub = pd.read_csv('submission.csv', index_col='id')
sub

: 

In [ ]:
!kaggle competitions submit -c playground-series-s4e8 -f submission.csv -m "[AUTOGLUON] Silver Rubanza, AUTOGLUON sub 3 custom with fastai ds  "

: 

# custom

In [5]:
target = 'class'

In [6]:
TIME_LIMIT = 3600 * 10

In [7]:
predictor = TabularPredictor(
    label=target,
    eval_metric='mcc',
    problem_type='binary',
    verbosity=2
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240824_160350"


In [ ]:
predictor.fit(
    train_data=train_df,
    time_limit=TIME_LIMIT,
    presets='best_quality',
    excluded_model_types=['KNN'],
    ag_args_fit={
        #'num_gpus': 1, 
        'stopping_metric': 'log_loss'
    }
)

baseline

# docs solution

In [8]:
target = 'class'
eval_metric = 'mcc'
train_data = train_df
Time_limit = 3600*10

In [6]:
%%time
#predictor = TabularPredictor(label='class').fit(train_data=train_df)

CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 4.53 µs


In [ ]:
#No path specified. Models will be saved in: "AutogluonModels/ag-20240823_201431"

In [9]:
%%time
predictor = TabularPredictor(label=target, eval_metric=eval_metric,verbosity=3).fit(
    train_data, presets='best_quality',excluded_model_types=['KNN'], time_limit=Time_limit
)

#results = predictor.fit_summary()

No path specified. Models will be saved in: "AutogluonModels/ag-20240824_160358"
Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #73~20.04.1-Ubuntu SMP Wed Jul 24 17:13:53 UTC 2024
CPU Count:          4
GPU Count:          0
Memory Avail:       11.03 GB / 15.34 GB (71.9%)
Disk Space Avail:   339.72 GB / 387.48 GB (87.7%)
Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': True, 'excluded_model_types': ['KNN'], 'num_bag_sets': 1}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': True,
 'calibrate': 'auto',
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
             'enable_ray_logging': True,
             'holdout_data': None,
  

(_ray_fit pid=4088) [50]	valid_set's binary_logloss: 0.176308	valid_set's mcc: 0.961319
(_ray_fit pid=4088) [100]	valid_set's binary_logloss: 0.0833169	valid_set's mcc: 0.973473 [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_ray_fit pid=4086) [150]	valid_set's binary_logloss: 0.0570748	valid_set's mcc: 0.977574 [repeated 4x across cluster]
(_ray_fit pid=4086) [200]	valid_set's binary_logloss: 0.0467027	valid_set's mcc: 0.980181 [repeated 4x across cluster]
(_ray_fit pid=4086) [250]	valid_set's binary_logloss: 0.0424726	valid_set's mcc: 0.981426 [repeated 4x across cluster]
(_ray_fit pid=4086) [300]	valid_set's binary_logloss: 0.0403701	valid_set's mcc: 0.982247 [repeated 4x across cluster]
(_ray_fit pid=4086) [350]	valid_set's binary_logloss: 0.0392204	valid_set's mcc: 0.982823 [repeated 4x across cluster]
(_r

(_ray_fit pid=12153) /home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/dask/dataframe/__init__.py:42: FutureWarning:  [repeated 4x across cluster]
(_ray_fit pid=12153) Dask dataframe query planning is disabled because dask-expr is not installed. [repeated 4x across cluster]
(_ray_fit pid=12153)  [repeated 8x across cluster]
(_ray_fit pid=12153) You can install it with `pip install dask[dataframe]` or `conda install dask`. [repeated 4x across cluster]
(_ray_fit pid=12153) This will raise in a future version. [repeated 4x across cluster]
(_ray_fit pid=12153)   warnings.warn(msg, FutureWarning) [repeated 4x across cluster]


(_ray_fit pid=4089) [2450]	valid_set's binary_logloss: 0.0359281	valid_set's mcc: 0.984582
(_ray_fit pid=4086) [2450]	valid_set's binary_logloss: 0.03599	valid_set's mcc: 0.984588 [repeated 2x across cluster]
(_ray_fit pid=4089) [2500]	valid_set's binary_logloss: 0.0359255	valid_set's mcc: 0.984588
(_ray_fit pid=4088) [2500]	valid_set's binary_logloss: 0.0357786	valid_set's mcc: 0.984844
(_ray_fit pid=4086) [2500]	valid_set's binary_logloss: 0.0359858	valid_set's mcc: 0.984553
(_ray_fit pid=12153) [50]	valid_set's binary_logloss: 0.174382	valid_set's mcc: 0.956458
(_ray_fit pid=4086) [2550]	valid_set's binary_logloss: 0.0359773	valid_set's mcc: 0.984565 [repeated 3x across cluster]
(_ray_fit pid=4088) [2600]	valid_set's binary_logloss: 0.0357627	valid_set's mcc: 0.984826
(_ray_fit pid=4089) [2600]	valid_set's binary_logloss: 0.0359149	valid_set's mcc: 0.984599
(_ray_fit pid=4086) [2600]	valid_set's binary_logloss: 0.0359746	valid_set's mcc: 0.984553
(_ray_fit pid=12153) [100]	valid_set

(_ray_fit pid=20346) /home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
(_ray_fit pid=20346) Dask dataframe query planning is disabled because dask-expr is not installed.
(_ray_fit pid=20346)  [repeated 2x across cluster]
(_ray_fit pid=20346) You can install it with `pip install dask[dataframe]` or `conda install dask`.
(_ray_fit pid=20346) This will raise in a future version.
(_ray_fit pid=20346)   warnings.warn(msg, FutureWarning)


...

(_ray_fit pid=76510) [850]	valid_set's binary_logloss: 0.035758	valid_set's mcc: 0.984965
(_ray_fit pid=72166) [2600]	valid_set's binary_logloss: 0.0368786	valid_set's mcc: 0.984419
(_ray_fit pid=76510) [900]	valid_set's binary_logloss: 0.035737	valid_set's mcc: 0.984966


(_ray_fit pid=77163) /home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
(_ray_fit pid=77163) Dask dataframe query planning is disabled because dask-expr is not installed.
(_ray_fit pid=77163) You can install it with `pip install dask[dataframe]` or `conda install dask`.
(_ray_fit pid=77163) This will raise in a future version.
(_ray_fit pid=77163)   warnings.warn(msg, FutureWarning)
(_ray_fit pid=77163)  [repeated 2x across cluster]


(_ray_fit pid=77115) [50]	valid_set's binary_logloss: 0.0826119	valid_set's mcc: 0.980351
(_ray_fit pid=76510) [950]	valid_set's binary_logloss: 0.0357333	valid_set's mcc: 0.984983
(_ray_fit pid=77115) [100]	valid_set's binary_logloss: 0.0416331	valid_set's mcc: 0.983051
(_ray_fit pid=76510) [1050]	valid_set's binary_logloss: 0.03573	valid_set's mcc: 0.985 [repeated 3x across cluster]
(_ray_fit pid=76510) [1100]	valid_set's binary_logloss: 0.0357172	valid_set's mcc: 0.985012 [repeated 3x across cluster]
(_ray_fit pid=77163) [150]	valid_set's binary_logloss: 0.0374869	valid_set's mcc: 0.983942
(_ray_fit pid=77115) [200]	valid_set's binary_logloss: 0.037721	valid_set's mcc: 0.983965
(_ray_fit pid=77163) [200]	valid_set's binary_logloss: 0.036506	valid_set's mcc: 0.984261 [repeated 2x across cluster]
(_ray_fit pid=76510) [1250]	valid_set's binary_logloss: 0.0357076	valid_set's mcc: 0.985018 [repeated 3x across cluster]
(_ray_fit pid=77163) [250]	valid_set's binary_logloss: 0.0361088	valid

KeyboardInterrupt: 

(_ray_fit pid=77163) [2650]	valid_set's binary_logloss: 0.0347917	valid_set's mcc: 0.985263


In [ ]:
#%%time
#predictor = TabularPredictor(label=target, eval_metric=eval_metric,verbosity=3).fit(
#    train_data,num_bag_folds=8, num_bag_sets=1, num_stack_levels=2, presets='best_quality', time_limit=Time_limit
#)

#results = predictor.fit_summary()

In [ ]:
#%%time
#predictor = TabularPredictor(label=target, eval_metric=eval_metric,verbosity=3).fit(
#    train_data,auto_stack=True, presets='best_quality', time_limit=Time_limit
#)

#results = predictor.fit_summary()

# Eval

In [10]:
results = predictor.fit_summary()

AssertionError: Predictor is not fit. Call `.fit` before calling `.fit_summary`.

(_ray_fit pid=77163) [2700]	valid_set's binary_logloss: 0.0347951	valid_set's mcc: 0.985269


In [11]:
%%time
predictions = predictor.predict(test_df)

AssertionError: Predictor is not fit. Call `.fit` before calling `.predict`.

(_ray_fit pid=77163) [2750]	valid_set's binary_logloss: 0.034804	valid_set's mcc: 0.985269
(_ray_fit pid=77163) [2800]	valid_set's binary_logloss: 0.0348047	valid_set's mcc: 0.985269


In [4]:
#predictor = TabularPredictor.load("AutogluonModels/ag-20240808_073211")

In [10]:
#predictor = TabularPredictor.load("AutogluonModels/ag-20240815_032551")


In [5]:
#predictor = TabularPredictor.load("AutogluonModels/ag-20240821_192353")



In [ ]:
predictions.shape

(2077964,)

In [10]:
%%time
predictor.evaluate(train_df)

Loading: AutogluonModels/ag-20240822_065518/models/KNeighborsDist_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240822_065518/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240822_065518/models/LightGBMXT_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240822_065518/models/LightGBM_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240822_065518/models/LightGBMXT_BAG_L2/model.pkl
Loading: AutogluonModels/ag-20240822_065518/models/LightGBM_BAG_L2/model.pkl
Loading: AutogluonModels/ag-20240822_065518/models/WeightedEnsemble_L3/model.pkl


CPU times: user 1h 30min 44s, sys: 17.6 s, total: 1h 31min 2s
Wall time: 45min 25s


{'mcc': 0.9855478037605251,
 'accuracy': 0.9928356130762653,
 'balanced_accuracy': 0.9928708224221436,
 'roc_auc': 0.9981602407113342,
 'f1': 0.9934465785070901,
 'precision': 0.9943976307430012,
 'recall': 0.9924973437254456}

In [ ]:
#predictor.evaluate(test_df)

In [15]:
%%time
predictor.leaderboard(train_df)

Loading: AutogluonModels/ag-20240822_065518/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240822_065518/models/KNeighborsDist_BAG_L1/model.pkl
Loading: AutogluonModels/ag-20240822_065518/models/LightGBMXT_BAG_L1/model.pkl


In [ ]:
predictor.leaderboard(extra_info=True)

In [ ]:
predictor.feature_importance(test_data)

In [ ]:
predictor.model_best

In [ ]:
predictor.model_names()

In [11]:
predictions

id
3116945    e
3116946    p
3116947    p
3116948    p
3116949    e
          ..
5194904    p
5194905    p
5194906    p
5194907    e
5194908    e
Name: class, Length: 2077964, dtype: object

In [ ]:
!rm submission.csv
!ls


rm: cannot remove 'submission.csv': No such file or directory
AutogluonModels  playground-series-s4e8		     wandb
lgbm_model.pkl	 playground-series-s4e8.zip	     xgb_model.json
main.py		 poisonous_mushrooms_classification
models		 requirements.txt


In [ ]:
#submit = pd.read_csv(path/'sample_submission.csv')
predictions = predictions.reset_index(drop=False)
submit = pd.DataFrame({'id': predictions['id'], 'class': predictions['class']})
submit.set_index('id', inplace=True)
submit.to_csv('submission.csv', index=True)
sub = pd.read_csv('submission.csv', index_col='id')
sub

,class
id,
3116945,e
3116946,p
3116947,p
3116948,p
3116949,e
...,...
5194904,p
5194905,p
5194906,p


In [ ]:

!kaggle competitions submit -c playground-series-s4e8 -f submission.csv -m "[AUTOGLUON] Silver Rubanza, AUTOGLUON sub 4 docs solution with presets  "

100%|██████████████████████████████████████| 19.8M/19.8M [00:00<00:00, 45.5MB/s]
Successfully submitted to Binary Prediction of Poisonous Mushrooms

In [13]:
# Get probabilities instead of class predictions
autogluon_probs = predictor.predict_proba(test_df)

In [15]:
xgb_optuna_params = {
              
    'num_leaves': 375, 
    'learning_rate': 0.0071851097672265855, 
    'n_estimators': 1851, 
    'subsample_for_bin': 104472,
    'min_child_samples': 454,
    'reg_alpha': 0.0011093389440775324,
    'reg_lambda': 0.15936937364526085,
    'colsample_bytree': 0.4533510196891779,
    'subsample': 0.9272807625455266,
    'max_depth': 20,
    'min_child_weight': 4,
    'gamma': 0.0033558210410357075,
    'max_leaves': 873,
    
    #'device': 'gpu'
}

In [17]:
%%time
xgb_model = xgb.XGBClassifier(**xgb_optuna_params)
xgb_model = xgb_model.fit(X_train, y_train)

xgb_preds = tensor(xgb_model.predict(test_dl.xs))

xgb_preds_x = tensor(xgb_model.predict(X_test))

xgb_score = matthews_corrcoef(y_test,xgb_preds_x)
xgb_score

NameError: name 'X_train' is not defined

In [18]:
# Get probabilities instead of class predictions
xgb_probs = xgb_model.predict_proba(X_test)

NameError: name 'X_test' is not defined

In [ ]:
import numpy as np

averaged_probs = (autogluon_probs + xgb_probs) / 2

In [ ]:
averaged_preds = np.argmax(averaged_probs, axis=1)

In [ ]:
mapping = dict(enumerate(dls.vocab))
submit = pd.read_csv(path/'sample_submission.csv')
submit['class'] = [mapping[pred] for pred in averaged_preds]
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')
print(sub)

In [8]:
#submit = pd.read_csv(path/'sample_submission.csv')
predictions = predictions.reset_index(drop=False)
submit = pd.DataFrame({'id': predictions['id'], 'class': predictions['class']})
submit.set_index('id', inplace=True)
submit.to_csv('submission.csv', index=True)
sub = pd.read_csv('submission.csv', index_col='id')
sub

,class
id,
3116945,e
3116946,p
3116947,p
3116948,p
3116949,e
...,...
5194904,p
5194905,p
5194906,p


In [ ]:
!ls

In [ ]:
submit = pd.read_csv(path/'sample_submission.csv')
predictions = predictions.reset_index(drop=False)
submit['class'] = predictions['class']
submit.to_csv('submission.csv', index=True)

In [ ]:
sub = pd.read_csv('submission.csv')
sub

,Unnamed: 0,id,class
0,0,3116945,e
1,1,3116946,p
2,2,3116947,p
3,3,3116948,p
4,4,3116949,e
...,...,...,...
2077959,2077959,5194904,p
2077960,2077960,5194905,p
2077961,2077961,5194906,p
2077962,2077962,5194907,e


In [9]:

!kaggle competitions submit -c playground-series-s4e8 -f submission.csv -m "[AUTOGLUON] Silver Rubanza, AUTOGLUON sub 3 custom  "

  0%|                                               | 0.00/19.8M [00:00<?, ?B/s]

100%|██████████████████████████████████████| 19.8M/19.8M [00:00<00:00, 48.0MB/s]
Successfully submitted to Binary Prediction of Poisonous Mushrooms

In [16]:
submit = pd.read_csv(path/'sample_submission.csv')
df = pd.DataFrame(predictions,columns=['class'])
df.head()
#submit['class']=df['class']
#submit.head()
#sol.to_csv('./[AutoML Grand Prix] Team GD Autogluon,mcc.csv',index=False)

,class
id,
3116945,e
3116946,p
3116947,p
3116948,p
3116949,e


In [17]:
submit['class']=df['class']
submit.head()

,id,class
0,3116945,NaN
1,3116946,NaN
2,3116947,NaN
3,3116948,NaN
4,3116949,NaN


In [ ]:

sol.to_csv('./[AutoML Grand Prix] Team GD Autogluon,mcc.csv',index=False)

In [13]:
submit = pd.read_csv(path/'sample_submission.csv')
submit['class'] = predictions
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')
sub

,id,class
0,3116945,NaN
1,3116946,NaN
2,3116947,NaN
3,3116948,NaN
4,3116949,NaN
...,...,...
2077959,5194904,NaN
2077960,5194905,NaN
2077961,5194906,NaN
2077962,5194907,NaN


In [ ]:
mapping = dict(enumerate(dls.vocab))
submit = pd.read_csv(path/'sample_submission.csv')
submit['class'] = [mapping[pred.item()] for pred in rounded_xgb_lgbm_nn_sub_preds]
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')
print(sub)

In [ ]:
# Make a copy of the training dataframe to avoid modifying the original
train_df_with_nan = train_df.copy()

# Add a NaN value to a random row in the 'stem-height' column
random_index = np.random.choice(train_df_with_nan.index)
train_df_with_nan.loc[random_index, 'stem-height'] = np.nan

# Verify the NaN was added
print(train_df_with_nan['stem-height'].isna().sum())